# Quantum machine learning on lower-dimensional single-cell RNAseq data

## Introduction

The general task of machine learning is to find and study patterns in data. Many machine learning algorithms map their input dataset to a higher dimensional feature space, through the use of a kernel function:

$$
k(\vec{x}_i, \vec{x}_j) = \langle f(\vec{x}_i), f(\vec{x}_j) \rangle
$$

where $k$ is the kernel function, $\vec{x}_i, \vec{x}_j$ are $n$ dimensional inputs, $f$ is a map from $n$-dimension to $m$-dimension space and $\langle a,b \rangle$ denotes the dot product. When considering finite data, a kernel function can be represented as a matrix:

$$
\textbf{K}_{ij} =  k(\vec{x}_i,\vec{x}_j)
$$

This notebook evaluates the following quantum machine learning models:

* Quantum Support Vector Machine (QSVC) https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.QSVC.html
* Pegasos QSVC: https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.PegasosQSVC.html
* Neural Networks: https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.NeuralNetworkClassifier.html
* Variational Quantum Classifier (VQC): https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.VQC.html


It takes as input the lower dimensional embedding of the single-cell RNAseq data with eight dimension of the melanoma minimal residual diseases sample and predicts drug-administered melanoma v/s phase II of minimal residual disease. 

In [11]:
# ====== Base class imports ======

import numpy as np
import pandas as pd
from glob import glob
import matplotlib
import os, time
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style('dark')

# ====== Scikit-learn imports ======
from sklearn.svm import SVC
from sklearn.metrics import (
    auc,
    roc_curve,
    ConfusionMatrixDisplay,
    f1_score,
    balanced_accuracy_score,
)
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split

# ====== Qiskit imports ======
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC, PegasosQSVC
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_algorithms.optimizers import COBYLA

## ====== Torch imports ======
import torch
import torch.nn as nn 
import pytorch_lightning as pl 
from torchmetrics.classification import F1Score
import torch.optim as optim
from lightning.pytorch.utilities.types import OptimizerLRScheduler
import torch.utils.data
from pytorch_lightning.loggers import TensorBoardLogger
import lightning, lightning.pytorch.loggers


### Load all checkpoints

In [12]:

all_checkpoints = []
for fname in glob('../SessionIII_MultiOmics/checkpoints/MRD/*.npy', recursive=True):
    all_checkpoints.append(fname)
print("Number of checkpoint files: ", len(all_checkpoints))

Number of checkpoint files:  4


In [13]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", 
#                                   token="4e7791327b95d470dcaae699d5f0a3073486fa3b22bd16c64bf65c609244728719c4389f587cac08285bd28488c69d1c68fc54ecbf4f160f2bff19ef04112846", 
#                                   set_as_default=True)


### Support Vector Classifier

We use a Support Vector Classifier (SVC) as a baseline to compare quantum machine learning methods. It uses the kernel function defined above to kernelize the data. 


In [14]:
def compute_svc(X_train, y_train, X_test, y_test, c = 1):
    beg_time = time.time()
    svc = SVC(C=c)
    svc_vanilla = svc.fit(X_train, y_train)
    labels_vanilla = svc_vanilla.predict(X_test)
    f1_svc = f1_score(y_test, labels_vanilla, average='micro')
    
    print("Time taken for SVC (secs): ", time.time() - beg_time)
    print("F1 SVC: ", f1_svc)
    
    return f1_svc

### Quantum Support Vector Classifier

In quantum machine learning, a quantum feature map, $\phi(\vec{x})$, maps a classical feature vector, $\vec{x}$, to a quantum Hilbert space, $| \phi(\vec{x})\rangle \langle \phi(\vec{x})|$. The quantum feature map transforms $\vec{x} \rightarrow | \phi(\vec{x})\rangle$ using a unitary transformation $\vec{U_\phi}(\vec{x})$, which is typically a [parameterized quantum circuit](./parameterized-quantum-circuits).

Constructing quantum feature maps based on parameterized quantum circuits that are hard to simulate classically is an important step towards possibly obtaining an advantage over classical machine learning approaches, and is an active area of current research.

In Reference 1, the authors propose a family of quantum feature maps that are [conjectured](gloss:conjecture) to be hard to simulate classically, and can be implemented as [short-depth](gloss:short-depth) circuits on [near-term quantum devices](gloss:near-term-quantum-devices). Qiskit implements these as the [`PauliFeatureMap`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.PauliFeatureMap.html). The quantum feature map of depth $d$ is implemented by the unitary operator:

$$ \mathcal{U}_{\Phi(\vec{x})}=\prod_d U_{\Phi(\vec{x})}H^{\otimes n},\ U_{\Phi(\vec{x})}=\exp\left(i\sum_{S\subseteq[n]}\phi_S(\vec{x})\prod_{k\in S} P_i\right)$$

which contains layers of Hadamard gates interleaved with [entangling blocks](gloss:entangling-blocks),

$U_{\Phi(\vec{x})}$, encoding the classical data as shown in circuit diagram below for $d=2$.

![](images/featuremap.svg)

Within the entangling blocks, $U_{\Phi(\vec{x})}$: $P_i \in \{ I, X, Y, Z \}$ denotes the [Pauli matrices](gloss:pauli-matrices), the index $S$ describes connectivity between different qubits or data points: $S \in \{\binom{n}{k}\ \mathrm{combinations,\ }k = 1,... n \}$, and by default the data mapping function $\phi_S(\vec{x})$ is

$$
\phi_S:\vec{x} \mapsto \Bigg\{
    \begin{array}{ll}
    x_i & \textit{if}\ S=\{i\} \\
    (\pi-x_i)(\pi-x_j) & \textit{if}\ S=\{i,j\}
    \end{array}
$$


when $k = 2, P_0 = Z, P_1 = ZZ$, this is the [`ZZFeatureMap`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html) in Qiskit:

$${\mathcal{U}_{\Phi(\vec{x})}} = \left( \exp\left(i\sum_{jk} \phi_{\{j,k\}}(\vec{x}) \, Z_j \otimes Z_k\right) \, \exp\left(i\sum_j \phi_{\{j\}}(\vec{x}) \, Z_j\right) \, H^{\otimes n} \right)^d$$

A generic Quantum machine learning pipeline will look like the following:

![](images/qml.png)

For different options of simulators look at https://qiskit.github.io/qiskit-aer/tutorials/1_aersimulator.html#Aer-Simulator-Options

In [24]:

    
def compute_QSVC(X_train, y_train, X_test, y_test, encoding='ZZ', c = 1, pegasos=False):
    beg_time = time.time()
    #service = QiskitRuntimeService(instance="accelerated-disc/internal/default")
    #backend =   service.backend('ibm_torino') 
    service = QiskitRuntimeService()    
    backend = AerSimulator(method='matrix_product_state')
    algorithm_globals.random_seed = 12345

    feature_map = None

    if encoding == 'ZZ' :
        feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], 
                            reps=2, 
                            entanglement='linear')
        #print("QSVC Circuit for ZZ Feature Map")
        #feature_map.draw('mpl')
    else: 
        if encoding == 'Z': 
            feature_map = ZFeatureMap(feature_dimension=X_train.shape[1], 
                            reps=2)
            print("QSVC Circuit for ZZ Feature Map")
            #feature_map.draw('mpl')
        if encoding == 'P': 
            feature_map = PauliFeatureMap(feature_dimension=X_train.shape[1], 
                            reps=2, entanglement='linear')
            print("QSVC Circuit for ZZ Feature Map")
            #feature_map.draw('mpl')

    sampler = Sampler() 
    fidelity = ComputeUncompute(sampler=sampler)
    
    Qkernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=Qkernel, C=c)
    qsvc_model = qsvc.fit(X_train,y_train)
    labels_qsvc = qsvc_model.predict(X_test)
    f1_qsvc = f1_score(y_test, labels_qsvc, average='micro')
    
    f1_peg_qsvc = 0
    if pegasos == True: 
        peg_qsvc = PegasosQSVC(quantum_kernel=Qkernel, C=c)
        peg_qsvc_model = peg_qsvc.fit(X_train, y_train)
        labels_peg_qsvc = peg_qsvc_model.predict(X_test)
        f1_peg_qsvc = f1_score(y_test, labels_peg_qsvc, average='micro')
        
    print("Time taken for QSVC (secs): ", time.time() - beg_time)
    if pegasos == False: 
        print("F1 QSVC: ", f1_qsvc)
    else: 
        print("F1 Pegasos QSVC: ", f1_peg_qsvc)
    
    return f1_qsvc,f1_peg_qsvc


### Quantum Neural Networks

In [25]:
def compute_QNN(X_train, y_train, X_test, y_test, primitive: str):
    beg_time = time.time()
    if primitive == 'estimator':
        # construct QNN with the QNNCircuit's default ZZFeatureMap feature map and RealAmplitudes ansatz.
        qc_qnn = QNNCircuit(num_qubits=X_train.shape[1])
        #print("Estimator QNN Circuit")
        #qc_qnn.draw('mpl')
        estimator_qnn = EstimatorQNN(circuit=qc_qnn)
        # QNN maps inputs to [-1, +1]
        estimator_qnn.forward(X_train[0, :], algorithm_globals.random.random(estimator_qnn.num_weights))
        # construct neural network classifier
        estimator_classifier = NeuralNetworkClassifier(estimator_qnn, optimizer=COBYLA(maxiter=100))
        # fit classifier to data
        estimator_classifier.fit(X_train, y_train)
        f1_score_estimator_qnn = estimator_classifier.score(X_test, y_test)
        print("Time taken for Sampler QNN (secs): ", time.time() - beg_time)
        print("F1 Estimator QNN: ", f1_score_estimator_qnn)
        
        return f1_score_estimator_qnn
    
    if primitive == 'sampler':
        # construct a quantum circuit from the default ZZFeatureMap feature map and a customized RealAmplitudes ansatz
        qc_sampler = QNNCircuit(ansatz=RealAmplitudes(X_train.shape[1], reps=1))
        #print("Sampler QNN Circuit")
        #qc_sampler.draw('mpl')
        # parity maps bitstrings to 0 or 1
        def parity(x):
            return "{:b}".format(x).count("1") % 2
        output_shape = 2  # corresponds to the number of classes, possible outcomes of the (parity) mapping
        # construct QNN
        sampler_qnn = SamplerQNN(circuit=qc_sampler, interpret=parity,output_shape=output_shape,)
        # construct classifier
        sampler_classifier = NeuralNetworkClassifier(neural_network=sampler_qnn, optimizer=COBYLA(maxiter=100))
        # fit classifier to data
        sampler_classifier.fit(X_train, y_train)
        f1_score_sampler_qnn = sampler_classifier.score(X_test, y_test)
        
        print("Time taken for Sampler QNN (secs): ", time.time() - beg_time)
        print("F1 Sampler QNN: ", f1_score_sampler_qnn)
        
        return f1_score_sampler_qnn

In [27]:
results_dict = {}
for i in range(1):
    matches = [x for x in all_checkpoints if "iter"+str(i)+"_" in x]
    x_train = np.load([x for x in matches if "train_embedding" in x][0])
    #print(x_train.shape)
    x_test = np.load([x for x in matches if "test_embedding" in x][0])
    #print(x_test.shape)
    y_train = np.load([x for x in matches if "train_labels" in x][0])
    #print(y_train.shape)
    y_test = np.load([x for x in matches if "test_labels" in x][0])
    #print(y_test.shape)

    
    f1_svc = compute_svc(x_train,
                        y_train,
                        x_test,
                        y_test,
                        c=10)
    
    f1_qsvc, f1_peg_qsvc = compute_QSVC(x_train, 
                                        y_train, 
                                        x_test,
                                        y_test,
                                        c=10,
                                        pegasos=0,
                                        )
    f1_est_qnn = compute_QNN(x_train, 
                            y_train, 
                            x_test, 
                            y_test, 
                            'estimator')
    
    f1_sam_qnn = compute_QNN(x_train, 
                            y_train, 
                            x_test, 
                            y_test, 
                            'sampler')
    
    results_dict[i] = [f1_svc, f1_qsvc, f1_peg_qsvc, f1_est_qnn, f1_sam_qnn]
    
df = pd.DataFrame.from_dict(results_dict, orient='index')
df.to_csv('PerformanceMetrics.csv', 
        index=False, 
        header=['SVC', 'QSVC', 'PEG_QSVC', 'EST_QNN', 'SAM_QNN'])

Time taken for SVC (secs):  0.010152339935302734
F1 SVC:  0.6
Time taken for QSVC (secs):  218.56338953971863
F1 QSVC:  0.6
Estimator QNN Circuit
Time taken for Sampler QNN (secs):  162.06342673301697
F1 Estimator QNN:  0.5666666666666667
Sampler QNN Circuit
Time taken for Sampler QNN (secs):  144.42334818840027
F1 Sampler QNN:  0.5666666666666667


## Results for predicting T0 vs MRD Phase 2 
![Res](qml_res.png "Quantum classificiaton of Minimal residual Disease for melanoma")